In [2]:
import os
import glob
import pickle
import ast
import json
import numpy as np
import seaborn as sns
import pandas as pd
from dance_evaluation import *
import matplotlib.pyplot as plt
# import analyze_hits_pos as anlyz

In [7]:
both_hand_df = pd.read_pickle(f"./one_sensor/tempo_60_140/pos/both_hand_x_zero_uni_60_140.pkl")
total = both_hand_df.shape[0]

# Load genre mappings
with open("genreID_count_mapping.json", "r") as file:
    genre_Tcount = json.load(file)

with open("genre_symbols_mapping.json", "r") as file:
    genre_name = json.load(file)

In [19]:
# key = "bpm_median"

bpm_dict = {"bpm_avg": 0,  "bpm_mode": 1, "bpm_median": 2}

a = 60; b= 140
metric = "pos"
mode = "zero_uni"
json_hits_data = {}
json_hits_genrewise = {}

for key in bpm_dict.keys():
    both_hand_x_score = pd.read_pickle(f"./one_sensor/tempo_{a}_{b}/{metric}/both_hand_x_{mode}_{a}_{b}_score.pkl")
    both_hand_y_score = pd.read_pickle(f"./one_sensor/tempo_{a}_{b}/{metric}/both_hand_y_{mode}_{a}_{b}_score.pkl")

    both_foot_x_score = pd.read_pickle(f"./one_sensor/tempo_{a}_{b}/{metric}/both_foot_x_{mode}_{a}_{b}_score.pkl")
    both_foot_y_score = pd.read_pickle(f"./one_sensor/tempo_{a}_{b}/{metric}/both_foot_y_{mode}_{a}_{b}_score.pkl")

    ##### X-AXIS
    both_hand_x_hits = set(both_hand_x_score.iloc[bpm_dict[key]]["hits_idx"])
    both_foot_x_hits = set(both_foot_x_score.iloc[bpm_dict[key]]["hits_idx"])

    both_hand_x_bpm = (both_hand_x_score.iloc[bpm_dict[key]]["bpm_hits_idx"])
    both_foot_x_bpm = (both_foot_x_score.iloc[bpm_dict[key]]["bpm_hits_idx"])

    ##### Y-AXIS
    both_hand_y_hits = set(both_hand_y_score.iloc[bpm_dict[key]]["hits_idx"])
    both_foot_y_hits = set(both_foot_y_score.iloc[bpm_dict[key]]["hits_idx"])

    both_hand_y_bpm = (both_hand_y_score.iloc[bpm_dict[key]]["bpm_hits_idx"])
    both_foot_y_bpm = (both_foot_y_score.iloc[bpm_dict[key]]["bpm_hits_idx"])


    ######## Final Hits Calculation ##########
    common_hits_x = both_hand_x_hits.intersection(both_foot_x_hits)
    union_hits_x = both_hand_x_hits.union(both_foot_x_hits)
    unique_both_hand_x_hits = set(both_hand_x_hits) - set(common_hits_x)
    unique_both_foot_x_hits = set(both_foot_x_hits) - set(common_hits_x)

    common_hits_y = both_hand_y_hits.intersection(both_foot_y_hits)
    union_hits_y = both_hand_y_hits.union(both_foot_y_hits)
    unique_both_hand_y_hits = set(both_hand_y_hits) - set(common_hits_y)
    unique_both_foot_y_hits = set(both_foot_y_hits) - set(common_hits_y)

    ########## hits id to BPM mapping ##############
    hand_id_to_bpms = {}
    foot_id_to_bpms = {}

    for id_val, bpm in zip(both_hand_y_hits, both_hand_y_bpm):
        hand_id_to_bpms[id_val] = bpm
    for id_val, bpm in zip(both_foot_y_hits, both_foot_y_bpm):
        foot_id_to_bpms[id_val] = bpm


    hit_stats = {"hits_idx": {"both_hand_x_hits": both_hand_x_hits, "both_foot_x_hits": both_foot_x_hits,
                            "both_hand_y_hits": both_hand_y_hits, "both_foot_y_hits": both_foot_y_hits,},
                
                "bpm_hits_idx": {"both_hand_x_bpm": both_hand_x_bpm, "both_foot_x_bpm": both_foot_x_bpm,
                                "both_hand_y_bpm": both_hand_y_bpm, "both_foot_y_bpm": both_foot_y_bpm,},
                
                "hits_count": {"both_hand_x": len(both_hand_x_hits), "both_foot_x": len(both_foot_x_hits),
                            "both_hand_y": len(both_hand_y_hits), "both_foot_y": len(both_foot_y_hits),},
                
                "final_hits_x": {"common_hits": common_hits_x, "union_hits": union_hits_x,
                                "unique_both_hand_x_hits": unique_both_hand_x_hits, 
                                "unique_both_foot_x_hits": unique_both_foot_x_hits},
                
                "final_hits_y": {"common_hits": common_hits_y, "union_hits": union_hits_y,
                                "unique_both_hand_y_hits": unique_both_hand_y_hits, 
                                "unique_both_foot_y_hits": unique_both_foot_y_hits},
                
                "hits_percentage": {"union_x": len(union_hits_x)*100/total,
                                    "union_y": len(union_hits_y)*100/total,
                                    "both_hand_x": len(both_hand_x_hits)*100/total, "both_foot_x": len(both_foot_x_hits)*100/total,
                                    "both_hand_y": len(both_hand_y_hits)*100/total, "both_foot_y": len(both_foot_y_hits)*100/total,},
                
                "estimated_bpm": {"both_hand_y": hand_id_to_bpms, "both_foot_y": foot_id_to_bpms},
                }
    json_hits_data[key] = hit_stats
    
    hits_list = {
    "union_x": json_hits_data[key]['final_hits_x']['union_hits'],
    "union_y": json_hits_data[key]['final_hits_y']['union_hits'],
    "both_hand_x": json_hits_data[key]["hits_idx"]['both_hand_x_hits'],
    "both_hand_y": json_hits_data[key]["hits_idx"]['both_hand_y_hits'],
    "both_foot_x": json_hits_data[key]["hits_idx"]['both_foot_x_hits'],
    "both_foot_y": json_hits_data[key]["hits_idx"]['both_foot_y_hits'],
    
        }

    # Initialize final dataframe with unique genres
    final_df = pd.DataFrame(list(genre_name.items()), columns=["dance_genre", "genre"])
    final_df["total"] = final_df["dance_genre"].map(genre_Tcount)

    # Dictionary to store genre-wise hit indices per limb
    genre_wise_hits = {limb: {} for limb in hits_list.keys()}

    # For each body segment, find genre-wise hit indices
    for limb, hit_indices in hits_list.items():
        hit_idx_list = list(hit_indices)
        hit_df = both_hand_df.iloc[hit_idx_list]                    # DataFrame for estimated hits of that limb

        # Group by genre and store indices per genre
        for genre_id, group in hit_df.groupby("dance_genre"):
            genre_wise_hits[limb][genre_id] = set(group.index)      # Store as set for easy intersection

        # groups the DataFrame hit_df by the column "dance_genre",counts how many rows belong to each genre, 
        # and creates a new column to store these counts.
        grouped = hit_df.groupby(['dance_genre']).size().reset_index(name=f'{limb}_count')
        final_df = final_df.merge(grouped, on='dance_genre', how='left')
        final_df[f'{limb}_count'] = final_df[f'{limb}_count'].fillna(0).astype(int)
        final_df[f'{limb}_percentage'] = round((final_df[f'{limb}_count'] / final_df['total']) * 100, 2)

    # Clean final_df to keep genre and counts
    hits_genrewise_df = final_df[['genre', 'total'] + 
                                [f'{limb}_count' for limb in hits_list.keys()] + 
                                [f'{limb}_percentage' for limb in hits_list.keys()]]
    
    json_hits_genrewise[key] = hits_genrewise_df
    

save_path1 = f"./one_sensor/tempo_60_140/{metric}/hit_stats_{mode}_{a}_{b}.pkl"
save_path2 = f"./one_sensor/tempo_60_140/{metric}/hit_stats_genrewise_{mode}_{a}_{b}.pkl"

with open(save_path1, "wb") as f:
    pickle.dump(json_hits_data, f)

with open(save_path2, "wb") as f:
    pickle.dump(json_hits_genrewise, f)
  
    